In [1]:
# Install software packages:
%pip install langchain-openai==0.2.0
%pip install python-dotenv==1.0.0
%pip install langchain==0.3.4
%pip install langchainhub==0.1.13
%pip install pypdf==3.17.0
%pip install tiktoken==0.5.1
%pip install chromadb
%pip install langchain-community==0.3.20
%pip install unstructured
%pip install python-docx
%pip install unstructured langchain-community
%pip install selenium
%pip install "unstructured[all-docs]"
%pip install gradio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached langchain-0.3.4-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.3.4-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.20 requires langchain<1.0.0,>=0.3.21, but you have langchain 0.3.4 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
  Using cached pypdf-3.17.0-py3-none-any.whl.metadata (7.5 kB)
Using cached pypdf-3.17.0-py3-none-any.whl (277 kB)
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.6.0
    Uninstalling pypdf-5.6.0:
      Successfully uninstalled pypdf-5.6.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.34.0 requires pypdf>=4.0, but you have pypdf 3.17.0 which is incompatible.


  Using cached tiktoken-0.5.1.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build tiktoken
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for tiktoken (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [50 lines of output]
      C:\Users\gscgo\AppData\Local\Temp\pip-build-env-p66vx_ce\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ************************************************

Note: you may need to restart the kernel to use updated packages.
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
Using cached langchain-0.3.25-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
Note: you may need to restart the kernel to use updated packages.
  Using cached pypdf-5.6.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.6.0-py3-none-any.whl (304 kB)
  Attempting uninstall: pypdf
    Found existing installation: pypdf 3.17.0
    Uninstalling pypdf-3.17.0:
      Successfully uninstalled pypdf-3.17.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updat

In [43]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

os.environ["ALLOW_RESET"] = "TRUE"

load_dotenv('.env', override=True)

# # Explicitly set the deployment name to match your Azure deployment
embedding_model = AzureOpenAIEmbeddings(
    model=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT'),
    azure_endpoint=os.getenv('AZURE_OPENAI_EMBEDDING_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_EMBEDDING_API_VERSION'),
    api_key=os.getenv('AZURE_OPENAI_EMBEDDING_API_KEY')
    )

# Instantiate an AzureOpenAI LLM:
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [71]:
#load web page from MOM PWM URLS:
from langchain.document_loaders import WebBaseLoader
loader_multiple_pages = WebBaseLoader(["https://www.mom.gov.sg/employment-practices/progressive-wage-model/cleaning-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/retail-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/food-services-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/occupational-pws-for-administrators-and-drivers",
                                      "https://www.mom.gov.sg/passes-and-permits/work-permit-for-foreign-worker/sector-specific-rules/non-traditional-sources-occupation-list"
                                      "https://www.mom.gov.sg/employment-practices/leave/shared-parental-leave"
                                      ])



#load CAs and legislations in docx and pdf format
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader

loader = DirectoryLoader("C:/Users/gscgo/OneDrive/Desktop/wst", show_progress=True, loader_cls=UnstructuredFileLoader)
data = loader.load()

100%|██████████| 82/82 [02:24<00:00,  1.76s/it]


In [72]:
# Split content into chunks.
# Import a splitter module called RecursiveCharacterTextSplitter:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Instantiate the splitter:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# Do the splitting:
splits = text_splitter.split_documents(loader.load())

# Embed and store splits in Chroma database.
# Import the Chroma and embeddings modules:
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# Instantiate the Chroma vector store with arguments:

# Comment out the following lines to process in smaller batches with delay to avoid hitting the rate limit when creating the embeddings:
# vectorstore = Chroma.from_documents(
#     documents=splits,
#     embedding=embedding_model
# )


import time
from tqdm import tqdm

# Create an empty Chroma DB first
vectorstore = Chroma(
    embedding_function=embedding_model
)

# Process in smaller batches
batch_size = 10
for i in tqdm(range(0, len(splits), batch_size)):
    batch = splits[i:i+batch_size]
    try:
        vectorstore.add_documents(documents=batch)
    except Exception as e:
        print(f"Error on batch {i}-{i+batch_size}: {e}")
        # Wait before retrying
        time.sleep(60)
        vectorstore.add_documents(documents=batch)
    # Small delay between successful batches
    time.sleep(2)

# Instantiate a retriever based on the vector store:
retriever = vectorstore.as_retriever() 

100%|██████████| 82/82 [00:59<00:00,  1.38it/s]
C:\Users\gscgo\AppData\Local\Temp\ipykernel_13424\598230684.py:26: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
100%|██████████| 1016/1016 [57:34<00:00,  3.40s/it] 


In [79]:
#Prompt Template
from langchain.schema.runnable import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import PromptTemplate
template = """
You are a virtual assistant for union officers who have to benchmark Employees Agreement to draft Agreements for negotiations.
For prompts relating to Progressive Wage Model or Occupational Progressive Wages, refer to the webpages loaded. For these prompts, to also include if the wage requirements refers monthly basic or gross wages.
Prompts with the word "PWM" refers to the Progressive Wage Model, while prompts with the word "OPW" refers to Occupational Progressive Wages. For these prompts, refer to the webpages loaded.For these prompts, to also include if the wage requirements refers to monthly basic or gross wages.
For prompts relating to 'Collective Agreements' or 'CAs', it refers to the Employees Agreements loaded. The names of the companies can be found in the first page of the Employees Agreement.
For prompts relating to legislations, refer to the PDF files loaded. 
For prompts relating to MyCareersFuture, scrape MyCareersFuture website for relevant information.
For prompts requesting to list down salaries of a specific job position across an industry (for example, hospitality, food manufacturing, food services, or wholesale and retail trade), refer to Employees Agreement of companies in those industries.To list down all the relevant job positions in all companies that belong to the industry.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Be as detailed as possible with your responses.


{context}

Question: {question}
Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

# Instantiate RAG chain:
custom_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
)

# Get completion:
completion = custom_rag_chain.invoke("What are the salary ranges for operators in food manufacturing CAs?")
print(completion.content)

In the food manufacturing sector, the salary ranges for operators as per the Collective Agreements (CAs) are as follows:

1. **Malaysia Dairy Industries (2023)**
   - Assistant Operator: Minimum S$1,250, Maximum S$1,800
   - Operator: Minimum S$1,350, Maximum S$1,950
   - Senior Operator: Minimum S$1,500, Maximum S$2,175

2. **Pokka Corp (2023)**
   - Production & Retort Operator I: Minimum S$1,200, Maximum S$1,900
   - Operator II: Minimum S$1,350, Maximum S$2,025

3. **SATS Food Services (2022)**
   - Production Operator: Minimum S$1,350, Maximum S$2,025
   - Senior Production Operator: Minimum S$1,500, Maximum S$2,250

These salary ranges refer to monthly basic wages.


In [2]:
# ========== COLLECTIVE AGREEMENT VIRTUAL ASSISTANT WITH GRADIO UI ==========
# This script creates a web-based Q&A interface using Gradio
# CHANGES MADE: Added Gradio UI components to transform console script into web app

import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
# ========== UPDATED IMPORTS (FIX FOR LANGCHAIN DEPRECATION) ==========
# CHANGE: Updated to use langchain_community for deprecated imports
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
# ========== NEW IMPORT FOR UI ==========
# CHANGE 1: Added Gradio import to enable web-based user interface. Remember to pip install gradio in the terminal within the virtual/conda environment before running the script.
import gradio as gr

import time
from tqdm import tqdm

os.environ["ALLOW_RESET"] = "TRUE"

load_dotenv('.env', override=True)

# # Explicitly set the deployment name to match your Azure deployment
embedding_model = AzureOpenAIEmbeddings(
    model=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT'),
    azure_endpoint=os.getenv('AZURE_OPENAI_EMBEDDING_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_EMBEDDING_API_VERSION'),
    api_key=os.getenv('AZURE_OPENAI_EMBEDDING_API_KEY')
    )

# Instantiate an AzureOpenAI LLM:
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

#load web page from MOM PWM URLS:
from langchain.document_loaders import WebBaseLoader
loader_multiple_pages = WebBaseLoader(["https://www.mom.gov.sg/employment-practices/progressive-wage-model/cleaning-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/retail-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/food-services-sector",
                                      "https://www.mom.gov.sg/employment-practices/progressive-wage-model/occupational-pws-for-administrators-and-drivers",
                                      "https://www.mom.gov.sg/passes-and-permits/work-permit-for-foreign-worker/sector-specific-rules/non-traditional-sources-occupation-list"
                                      "https://www.mom.gov.sg/employment-practices/leave/shared-parental-leave"
                                      ])



#load CAs and legislations in docx and pdf format
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader

loader = DirectoryLoader("C:/Users/gscgo/OneDrive/Desktop/wst", show_progress=True, loader_cls=UnstructuredFileLoader)
data = loader.load()



# Split content into chunks.
# Import a splitter module called RecursiveCharacterTextSplitter:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Instantiate the splitter:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# Do the splitting:
splits = text_splitter.split_documents(loader.load())

# Embed and store splits in Chroma database.
# Import the Chroma and embeddings modules:
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# Instantiate the Chroma vector store with arguments:

# Comment out the following lines to process in smaller batches with delay to avoid hitting the rate limit when creating the embeddings:
# vectorstore = Chroma.from_documents(
#     documents=splits,
#     embedding=embedding_model
# )


import time
from tqdm import tqdm

# Create an empty Chroma DB first
vectorstore = Chroma(
    embedding_function=embedding_model
)

# Process in smaller batches
batch_size = 10
for i in tqdm(range(0, len(splits), batch_size)):
    batch = splits[i:i+batch_size]
    try:
        vectorstore.add_documents(documents=batch)
    except Exception as e:
        print(f"Error on batch {i}-{i+batch_size}: {e}")
        # Wait before retrying
        time.sleep(60)
        vectorstore.add_documents(documents=batch)
    # Small delay between successful batches
    time.sleep(2)

# Instantiate a retriever based on the vector store:
retriever = vectorstore.as_retriever() 

# ========== MINOR MODIFICATION FOR UI ==========
# Set up a retrieval chain:
# CHANGE 2: Kept return_source_documents=True to show source citations in UI
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5}
    ),
    return_source_documents=True  # Essential for source citations in web UI
)

# ========== MAJOR CHANGE: REPLACED HARDCODED EXECUTION WITH INTERACTIVE FUNCTION ==========
# ORIGINAL CODE REMOVED:
# query = "In the event that retrenchment is inevitable, what should companies do?"
# completion = qa.invoke({"query": query})
# print(completion['result'])

# CHANGE 3: Created interactive function to handle user questions
def ask_question(question):
    """
    Function to process user questions and return answers with sources
    
    CHANGES MADE FOR GRADIO UI:
    - Takes user input instead of hard-coded query
    - Validates input (checks for empty questions)  
    - Formats response with source citations for better web display
    - Includes error handling for robust UI experience
    - Returns formatted string suitable for Gradio text display
    """
    if not question.strip():
        return "Please enter a question."
    
    try:
        # Run the retrieval chain and get a response from the LLM
        completion = qa.invoke({"query": question})
        
        # Format the response with sources
        answer = completion['result']
        sources = completion.get('source_documents', [])
        
        # NEW FEATURE: Add source citations to answer (not in original script)
        if sources:
            source_info = "\n\n**Sources:**\n"
            for i, doc in enumerate(sources[:3], 1):  # Show top 3 sources
                source_info += f"{i}. {doc.metadata.get('source', 'Unknown source')}\n"
            answer += source_info
        
        return answer
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
#selenium for MyCareersFuture scraping
    import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json

def get_salary_info(job_role, headless=True, timeout=20):
    """
    Scrapes MyCareersFuture website for salary information for a given job role.

    Args:
        job_role (str): The job role to search for (e.g., 'Data Scientist', 'Software Engineer')
        headless (bool): Whether to run the browser in headless mode (no UI)
        timeout (int): Maximum time to wait for elements to load in seconds

    Returns:
        dict: Dictionary containing salary information for the job role
    """
    # Set up Selenium webdriver with Chrome
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless=new")  # Using newer headless mode
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Navigate to MyCareersFuture
        driver.get("https://www.mycareersfuture.gov.sg/")
        print(f"Searching for salary information for: {job_role}")

        # Wait for page to load completely
        time.sleep(3)

        # Print page title for debugging
        print(f"Page title: {driver.title}")

        # Try multiple selectors for the search box
        search_box = None
        selectors = [
            "input[placeholder='Search for keywords']",
            "input[type='text']",
            "input.search-input",
            "//input[@placeholder='Search for keywords']",
            "//input[contains(@class, 'search')]"
        ]

        for selector in selectors:
            try:
                if selector.startswith("//"):
                    search_box = WebDriverWait(driver, timeout).until(
                        EC.element_to_be_clickable((By.XPATH, selector))
                    )
                else:
                    search_box = WebDriverWait(driver, timeout).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
                    )
                print(f"Found search box using selector: {selector}")
                break
            except Exception as e:
                print(f"Selector {selector} failed: {e.__class__.__name__}")
                continue

        if not search_box:
            # Take screenshot for debugging
            driver.save_screenshot("debug_screenshot.png")
            print("Could not find search box. Screenshot saved as debug_screenshot.png")
            print("HTML source snippet:")
            print(driver.page_source[:500] + "...")
            return {"error": "Could not find search input on the website"}

        # Continue with the search
        search_box.clear()
        search_box.send_keys(job_role)
        search_box.send_keys(Keys.RETURN)

        # Wait for search results to load
        print("Waiting for search results...")
        try:
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR,
                    "[data-testid='job-card'], .search-card, .job-card, article, div[class*='JobCard']"))
            )
            print("Search results loaded")
            # Take screenshot for debugging
            driver.save_screenshot("search_results.png")
            print("Search results screenshot saved")
        except Exception as e:
            print(f"Error waiting for search results: {e}")
            driver.save_screenshot("search_results_debug.png")
            return {"error": f"Could not find search results for {job_role}"}

        # Extract salary information from the search results
        salary_data = {
            "job_role": job_role,
            "salary_range": [],
            "average_salary": None,
            "median_salary": None,
            "sample_size": 0
        }

        # Try to find a salary indicator or insights section
        print("Looking for salary insights...")
        try:
            # Check if there's a salary insights section
            salary_elements = driver.find_elements(By.XPATH,
                "//*[contains(text(), 'Salary')]/ancestor::div[3]")

            if salary_elements:
                print(f"Found {len(salary_elements)} potential salary elements")
                for el in salary_elements:
                    print(f"Salary element text: {el.text[:100]}...")
                    if "$" in el.text:
                        # Extract all salary ranges using regex
                        salary_matches = re.findall(r'\$[\d,]+\s*-\s*\$[\d,]+|\$[\d,]+', el.text)
                        if salary_matches:
                            for match in salary_matches:
                                if match not in salary_data["salary_range"]:
                                    salary_data["salary_range"].append(match)
                            print(f"Extracted salary ranges: {salary_matches}")

                        # Try to find average and median
                        if "average" in el.text.lower():
                            avg_match = re.search(r'average.*?(\$[\d,]+)', el.text.lower())
                            if avg_match:
                                salary_data["average_salary"] = avg_match.group(1)
                                print(f"Found average salary: {avg_match.group(1)}")

                        if "median" in el.text.lower():
                            med_match = re.search(r'median.*?(\$[\d,]+)', el.text.lower())
                            if med_match:
                                salary_data["median_salary"] = med_match.group(1)
                                print(f"Found median salary: {med_match.group(1)}")
        except Exception as e:
            print(f"Error processing salary insights: {e}")

        # If no salary insights found, try job cards
        if not salary_data["salary_range"]:
            print("Looking for salary info in job cards...")

            # Try multiple selectors for job cards
            job_card_selectors = [
                "[data-testid='job-card']",
                ".search-card",
                ".job-card",
                "article",
                "div[class*='JobCard']",
                "div[class*='job-card']",
                "//div[contains(@class, 'job') and contains(@class, 'card')]"
            ]

            job_cards = []
            for selector in job_card_selectors:
                try:
                    if selector.startswith("//"):
                        cards = driver.find_elements(By.XPATH, selector)
                    else:
                        cards = driver.find_elements(By.CSS_SELECTOR, selector)

                    if cards:
                        print(f"Found {len(cards)} job cards using selector: {selector}")
                        job_cards = cards
                        break
                except Exception as e:
                    print(f"Job card selector {selector} failed: {e.__class__.__name__}")

            # Process job cards for salary information
            for card in job_cards[:15]:  # Check first 15 job postings
                try:
                    card_text = card.text
                    print(f"Job card text snippet: {card_text[:50]}...")

                    # Try to extract salary using regex
                    salary_matches = re.findall(r'\$[\d,]+\s*-\s*\$[\d,]+|\$[\d,]+', card_text)
                    if salary_matches:
                        for match in salary_matches:
                            if match not in salary_data["salary_range"]:
                                salary_data["salary_range"].append(match)
                        print(f"Extracted salary ranges: {salary_matches}")
                except Exception as e:
                    print(f"Error processing job card: {e.__class__.__name__}")
                    continue

            if salary_data["salary_range"]:
                salary_data["sample_size"] = len(salary_data["salary_range"])
                print(f"Found {len(salary_data['salary_range'])} job postings with salary information")

        # If still no salary info, try one more approach - scan entire page for salary patterns
        if not salary_data["salary_range"]:
            print("Scanning entire page for salary information...")
            page_text = driver.page_source
            salary_matches = re.findall(r'\$[\d,]+\s*-\s*\$[\d,]+|\$[\d,]+', page_text)
            if salary_matches:
                # Filter out duplicates
                unique_salaries = list(set(salary_matches))
                salary_data["salary_range"] = unique_salaries
                salary_data["sample_size"] = len(unique_salaries)
                print(f"Found {len(unique_salaries)} salary mentions on the page")

        # Calculate average and median from the collected salary ranges
        if salary_data["salary_range"]:
            salary_values = []

            for salary_str in salary_data["salary_range"]:
                # Handle ranges like "$5,000 - $8,000"
                if "-" in salary_str:
                    parts = salary_str.split("-")
                    min_sal = float(parts[0].replace("$", "").replace(",", "").strip())
                    max_sal = float(parts[1].replace("$", "").replace(",", "").strip())
                    # Use the midpoint of the range
                    salary_values.append((min_sal + max_sal) / 2)
                else:
                    # Handle single values like "$5,000"
                    salary_values.append(float(salary_str.replace("$", "").replace(",", "").strip()))

            if salary_values:
                # Calculate average
                avg_salary = sum(salary_values) / len(salary_values)
                salary_data["average_salary"] = f"${int(avg_salary):,}"

                # Calculate median
                sorted_values = sorted(salary_values)
                mid = len(sorted_values) // 2
                if len(sorted_values) % 2 == 0:
                    median_salary = (sorted_values[mid-1] + sorted_values[mid]) / 2
                else:
                    median_salary = sorted_values[mid]
                salary_data["median_salary"] = f"${int(median_salary):,}"

                print(f"Calculated average salary: {salary_data['average_salary']}")
                print(f"Calculated median salary: {salary_data['median_salary']}")

        return salary_data

    except Exception as e:
        print(f"Unexpected error: {e}")
        driver.save_screenshot("error_screenshot.png")
        return {"error": str(e)}
    finally:
        # Clean up
        driver.quit()

# ========== CHANGE 4: COMPLETE GRADIO WEB INTERFACE (ENTIRELY NEW) ==========
# This entire section replaces the simple print statement with a full web UI
with gr.Blocks(title="Collective Agreement Virtual Assistant") as app:
    # Professional header with description
    gr.Markdown(
        """
        # 🤖 Collective Agreement Virtual Assistant
        
        Ask questions about collective agreements, salary ranges, and progressive wage models.
        The assistant draws information from existing legislations, tripartite guidelines, CA documents, and MOM resources.
        """
    )
        
    # Two-column layout for clean interface
    with gr.Row():
        with gr.Column():
            # Question input area with helpful placeholder
            question_input = gr.Textbox(
                label="Your Question",
                placeholder="e.g., What is the salary of bellman across food manufacturing CAs?",
                lines=3
            )
            # Primary submit button
            submit_btn = gr.Button("Ask Question", variant="primary")
            
        with gr.Column():
            # Answer display area with appropriate sizing
            answer_output = gr.Textbox(
                label="Answer",
                lines=15,
                max_lines=20
            )
    
    # Example questions to guide users
    gr.Examples(
        examples=[
            ["What is the salary ranges for operator across food manufactuirng CAs?"],
            ["What is the current progressive wage model wage requirements for restroom cleaners?"],
            ["What is the salary ranges for housekeepers across hotels CAs?"],
            ["What is the current legislation for shared parental leave?"]
        ],
        inputs=question_input
    )
    
    # Connect the interactive function to UI elements
    # Two ways to submit questions for better user experience
    submit_btn.click(
        fn=ask_question,
        inputs=question_input,
        outputs=answer_output
    )
    
    # Allow Enter key submission
    question_input.submit(
        fn=ask_question,
        inputs=question_input,
        outputs=answer_output
    )

#PROMPT TEMPLATE
from langchain.schema.runnable import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import PromptTemplate
template = """
You are a virtual assistant for union officers who have to benchmark Employees Agreement to draft Agreements for negotiations.
For prompts relating to Progressive Wage Model or Occupational Progressive Wages, refer to the webpages loaded. For these prompts, to also include if the wage requirements refers monthly basic or gross wages.
Prompts with the word "PWM" refers to the Progressive Wage Model, while prompts with the word "OPW" refers to Occupational Progressive Wages. For these prompts, refer to the webpages loaded.For these prompts, to also include if the wage requirements refers to monthly basic or gross wages.
For prompts relating to 'Collective Agreements' or 'CAs', it refers to the Employees Agreements loaded. The names of the companies can be found in the first page of the Employees Agreement.
For prompts relating to legislations, refer to the PDF files loaded. 
For prompts relating to MyCareersFuture, scrape MyCareersFuture website for relevant information.
For prompts requesting to list down salaries of a specific job position across an industry (for example, hospitality, food manufacturing, food services, or wholesale and retail trade), refer to Employees Agreement of companies in those industries.To list down all the relevant job positions in all companies that belong to the industry.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Be as detailed as possible with your responses.


{context}

Question: {question}
Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

# Instantiate RAG chain:
custom_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
)

# ========== CHANGE 5: LAUNCH WEB INTERFACE INSTEAD OF ONE-TIME EXECUTION ==========
# REPLACED: Single execution with print output
# NEW: Persistent web server that stays active for continuous use
if __name__ == "__main__":
    # share=True creates a public shareable link
    # Server runs continuously until stopped
    app.launch(share=True)

USER_AGENT environment variable not set, consider setting it to identify your requests.
100%|██████████| 82/82 [00:51<00:00,  1.58it/s]
C:\Users\gscgo\AppData\Local\Temp\ipykernel_8072\1316093069.py:91: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
100%|██████████| 1016/1016 [1:44:43<00:00,  6.18s/it]    


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://3e5bc193c5b3598c21.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
